![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

# Módulo 5: Aprendizaje Supervisado. Proyecto final.
## Probabilidad de impago financiero.

* [A. Introducción](#introduction)
    * [A.1. Conjunto de datos](#dataset)
    * [A.1. Objetivo](#objetive)
* [B. Importar librerías](#libraries)
* [C. Importar datos](#data)


* [1. Entendimiento de los datos](#data_understanding)
    * [1.1: Análisis univariado de datos](#univariate_data_analysis)
        * [1.1.1. Tamaño del dataset](#dataset_size)
        * [1.1.2. Visualización directa de los datos](#direct_visualization)
        * [1.1.3. Tipos de las variables](#variable_types)
        * [1.1.4. Estadística descriptiva](#descriptive_statistics)
        * [1.1.5. Valores nulos](#null_values)
        * [1.1.6. Distribución de la variable target](#target_distribution)
        * [1.1.7-2.1.10. Variables distribution & relations with target](#variables_distribution)
    * [1.2. MLC2.2.: Analisis multivariado de los datos](#multivariate_data_analysis)

 <!--- 
        
* [3. MLC3: Data preparation](#data_preparation)
    * [3.1. MLC 3.1. Data cleaning](#data_cleaning)
        * [3.1.1. Dealing with variable types](#dealing_variable_types)
        * [3.1.2. Imputation of null values](#nulls_imputation)
        * [3.1.4. Elimination of features with low variance](#low_variance)
    * [3.2. MLC 3.2. Data transformation](#data_transformation)
        * [3.2.1. Transformation of categorical variables](#transformation_categorical)
    * [3.3. MLC 3.3. Feature engineering](#feature_engineering)



* [4. MLC4: Modelling](#modelling)
    * [4.1. MLC 4.1. Data partition and sampling](#dataset_partition)
        * [Random hold hout](#random_holdout)
        * [K-fold cross-validation](#k_fold)
    * [4.2. MLC 4.2. Model definition](#model_definition)
    * [4.3. MLC 4.3. Model optimization](#model_optimization)
        * [4.3.1. Hyperparameter optimization](#hyperparameter_optimization)
        * [4.3.2. Ensemble models](#ensemble_models)


* [5. MLC5: Evaluation](#model_evaluation)
-->


## A. Introducción <a class="anchor" id="introduction"></a>

En este proyecto usaremos el Aprendizaje Supervisado para la predicción de impagos financieros.

### Conjuto de datos <a class="anchor" id="dataset"></a>
Usaremos una competición de Kaggle, la cuál podemos acceder desde el siguiente enlace:

https://www.kaggle.com/c/home-credit-default-risk/

### Objetivo <a class="anchor" id="objetive"></a>
Desarrollar un proyecto de modelización supervisada para predecir la probabilidad de impago financiero.

El trabajo debe incluir todas las etapas contempladas en la checklist de ML vista en la sesión 1:
1. **Entendimiento de los datos** 
2. **Preparación de los datos**
3. **Modelización**
4. **Evaluación**

El código tendrá que estar correctamente organizado y comentado para facilitar su comprensión, y deberá poder ejecutarse sin errores. No hay restricciones en cuanto al uso de las técnicas y algoritmos supervisados que se consideren.

## B. Importar librerías<a id="libraries"></a>
Primero vamos a importar 3 librerías utilizadas para el Análisis de Datos en ML:

In [22]:
# Librerías para trabajar con datos (Series y DataFrames)
import numpy as np
import pandas as pd

# Librería de visualización
import plotly.express as px

# Librería de análisis exploratorio
from pandas_profiling import ProfileReport

También usaremos la librería de preprocesamiento de **scikit-learn**.

In [23]:
from sklearn import preprocessing

Realizamos algunos ajustes para corregir las visualizaciónes

In [24]:
# Ajustamos las opciones de pandas para que nos devuelva todas las filas que queremos visualizar
pd.set_option('display.max_rows', 150)

# Ajustamos para que los valores float se muestren con dos decimales
pd.options.display.float_format = '{:,.2f}'.format

## C. Importar datos <a id="data"></a>
Antes de comenzar con la Checklist del proyecto, importaremos los datos. El conjunto de datos está disponible en el enlace de Kaggle indicado en el apartado anterior ['Conjunto de datos'](#dataset). 

En este caso sólo usaremos dos dataset: **application_train.csv** y **application_test.csv**:
* Ambos son los datos principales, divididos en dos archivos para el entreno del modelo (con TARGET) y el testeo (sin TARGET).
* Una fila representa un préstamo en nuestra muestra de datos.

Uniremos los dos dataframes y le asignaremos un valor 'UNK' en la variable target. Así podemos hacer la parte de Análisis de Datos (limpieza, visualización y preprocesamiento) con todo el conjunto de datos.

In [25]:
%%time
# Para resetear el index le asignamos la primera columna y luego el método reset_index.
df_train = pd.read_csv('./data/application_train.csv', index_col=0).reset_index()
df_test = pd.read_csv('./data/application_test.csv', index_col=0).reset_index()

In [ ]:
# Identificamos el nombre de la columna utilizado en el DF Train (nombre de la columna en train que no está en test)
[col for col in df_train.columns.to_list() if col not in df_test.columns.to_list()]

['TARGET']

In [ ]:
# Creamos la columna TARGET en df_test
df_test['TARGET'] = 'UNK'

In [ ]:
# Comprobamos que los dos DF tienen las mismas columnas
assert [col for col in df_train.columns.to_list() if col not in df_test.columns.to_list()] == []

In [ ]:
# Unimos los dos DFs
df = pd.concat([df_train, df_test]).reset_index(drop=True)

In [ ]:
# Comprobamos que las filas de df es la suma de las filas de train y test
assert len(df_train)+len(df_test) == len(df)
# Comprobamos que tienen el mísmo numero de columnas
assert df_train.shape[1] == df_test.shape[1] == df.shape[1]

***

# 1. Entendimiento de los datos <a id="data_understanding"></a>

## 1.1: Análisis univariado de los datos <a id="univariate_data_analysis"></a>

### 1.1.1. Tamaño del dataset <a id="dataset_size"></a>

In [ ]:
df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(40), object(17)
memory usage: 331.6+ MB


In [ ]:
# Mostramos la forma del dataset
df.shape

(356255, 122)

### 1.1.2.  Visualización directa de los datos <a id="direct_visualization"></a>

In [ ]:
# Visualizamos 3 muestras aleatorias del dataset
df.sample(3).T

,46766,226593,210374
SK_ID_CURR,154170,362470,343788
TARGET,0,0,0
NAME_CONTRACT_TYPE,Cash loans,Cash loans,Cash loans
CODE_GENDER,M,M,F
FLAG_OWN_CAR,N,Y,N
FLAG_OWN_REALTY,N,Y,N
CNT_CHILDREN,0,1,0
AMT_INCOME_TOTAL,"225,000.00","202,500.00","157,500.00"
AMT_CREDIT,"1,078,200.00","970,380.00","1,350,000.00"
AMT_ANNUITY,"38,331.00","28,372.50","39,474.00"


### 1.1.3. Tipos de las variables<a id="variable_types"></a>

In [ ]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Data columns (total 122 columns):
 #    Column                        Dtype  
---   ------                        -----  
 0    SK_ID_CURR                    int64  
 1    TARGET                        object 
 2    NAME_CONTRACT_TYPE            object 
 3    CODE_GENDER                   object 
 4    FLAG_OWN_CAR                  object 
 5    FLAG_OWN_REALTY               object 
 6    CNT_CHILDREN                  int64  
 7    AMT_INCOME_TOTAL              float64
 8    AMT_CREDIT                    float64
 9    AMT_ANNUITY                   float64
 10   AMT_GOODS_PRICE               float64
 11   NAME_TYPE_SUITE               object 
 12   NAME_INCOME_TYPE              object 
 13   NAME_EDUCATION_TYPE           object 
 14   NAME_FAMILY_STATUS            object 
 15   NAME_HOUSING_TYPE             object 
 16   REGION_POPULATION_RELATIVE    float64
 17   DAYS_BIRTH                    int64  
 18   DA

### 1.1.4. Estadística descriptiva <a id="descriptive_statistics"></a>

In [ ]:
# Análisis de la distribución de las variables numéricas
df.describe().T

,count,mean,std,min,25%,50%,75%,max
SK_ID_CURR,"356,255.00","278,128.00","102,842.10","100,001.00","189,064.50","278,128.00","367,191.50","456,255.00"
CNT_CHILDREN,"356,255.00",0.41,0.72,0.00,0.00,0.00,1.00,20.00
AMT_INCOME_TOTAL,"356,255.00","170,116.06","223,506.82","25,650.00","112,500.00","153,000.00","202,500.00","117,000,000.00"
AMT_CREDIT,"356,255.00","587,767.41","398,623.69","45,000.00","270,000.00","500,211.00","797,557.50","4,050,000.00"
AMT_ANNUITY,"356,219.00","27,425.56","14,732.81","1,615.50","16,731.00","25,078.50","34,960.50","258,025.50"
AMT_GOODS_PRICE,"355,977.00","528,020.00","366,064.99","40,500.00","234,000.00","450,000.00","675,000.00","4,050,000.00"
REGION_POPULATION_RELATIVE,"356,255.00",0.02,0.01,0.00,0.01,0.02,0.03,0.07
DAYS_BIRTH,"356,255.00","-16,041.25","4,358.80","-25,229.00","-19,676.00","-15,755.00","-12,425.00","-7,338.00"
DAYS_EMPLOYED,"356,255.00","64,317.23","141,705.53","-17,912.00","-2,781.00","-1,224.00",-290.00,"365,243.00"
DAYS_REGISTRATION,"356,255.00","-4,983.59","3,526.97","-24,672.00","-7,477.00","-4,502.00","-1,995.00",0.00


In [ ]:
# Excluimos las variables numéricas para ver la distribución del resto de variables
df.describe(exclude=['int', 'float']).T

,count,unique,top,freq
TARGET,356255,3,0,282686
NAME_CONTRACT_TYPE,356255,2,Cash loans,326537
CODE_GENDER,356255,3,F,235126
FLAG_OWN_CAR,356255,2,N,235235
FLAG_OWN_REALTY,356255,2,Y,246970
NAME_TYPE_SUITE,354052,7,Unaccompanied,288253
NAME_INCOME_TYPE,356255,8,Working,183307
NAME_EDUCATION_TYPE,356255,5,Secondary / secondary special,252379
NAME_FAMILY_STATUS,356255,6,Married,228715
NAME_HOUSING_TYPE,356255,6,House / apartment,316513


In [ ]:
profile=ProfileReport(df)

In [ ]:
profile

Summarize dataset:  51%|█████     | 65/127 [00:15<00:02, 24.82it/s, Describe variable:ENTRANCES_MODE]              

: 

: 

In [17]:
pandas_profiling.__version__

NameError: name 'pandas_profiling' is not defined